<center>
    <br>
    <font size="14"><b>Documentation Développeur</b></font>
    <br>
    <br>
    <font size="4">Logiciel de visualisation des résidus DORIS IGN/IPGP/JPL</font><br>
    <font size="4">Vanessa Monnier - vanessa.monnier@ensg.eu</font><br>
    <font size="2">novembre 2024 - janvier 2025</font>
</center>
Développée par Vanessa Monnier, étudiante ingénieure filière PPMD, sous l'encadrement de Arnaud Pollet, Samuel Nahmani, Guillaume Lion

## I - Description <a class="anchor" id="Q1"></a>
Ce projet consiste à développer un logiciel de visualisation des résidus de traitement des mesures DORIS et des produits associés (résidus d’observation, paramètres orbitaux) fournis par l'International Doris Service. Cet outil a pour but d'être utilisé, et que sont développement soit poursuivit, par le centre d'analyse IGN/IPGP/JPL. Il utilise une architecture MVC (modèle, vue, controlleur) et implémente divers design patterns (Strategy, Factory, Observer). Cet architecture a pour but de rendre ce projet accessible et facile de prise en main de code, pour un prochain étudiant qui sera chargé de continuer le développement de l'outil.

Le logiciel permet actuellement de visualiser les positions géographiques des stations sur une carte mondiale, et d'afficher plusieurs graphiques (skyplot, graphique d'évolution des résidus).



<br>
    <font size="4"><b>Sommaire</b></font><br>
    
* [I - Description](#Q1)
* [II - Modules Python à importer](#Q2)
* [III - Jeu de donnée](#Q3)
* [III - Explications des différentes classes](#Q4)
  * [Classe Modele]
  * [Classe Controleur ]
  * [Classe Vue ]
* [Problèmes rencontrés](#Q5)
* [Contributions](#Q6)



## II - Modules Python à importer <a class="anchor" id="Q2"></a>

### a) Modules à importer pour la partie "View"

In [1]:
from abc import ABC, abstractmethod
from PyQt5.QtWidgets import QMainWindow, QWidget, QVBoxLayout, QToolBar, QLabel, QAction, QLineEdit, QComboBox, QPushButton, QHBoxLayout, QCheckBox, QScrollArea, QListWidget, QListWidgetItem, QAbstractItemView, QMessageBox, QFileDialog, QDesktopWidget
from PyQt5.QtWebEngineWidgets import QWebEngineView
import folium
from PyQt5.QtCore import QUrl, pyqtSignal, QPoint, Qt
import os
from PyQt5 import QtCore

from PyQt5.QtCore import Qt, QObject
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure

import numpy as np
import matplotlib.pyplot as plt
import random
#pip install "nom_librairie" si problème d'import

### b) Modules à importer pour la partie "Controleur"

In [2]:
from Modele import Modele
import View
from View import fenetre_Skyplot, Skyplot
from PyQt5.QtWidgets import QMainWindow, QWidget, QVBoxLayout, QToolBar, QLabel, QAction, QLineEdit, QComboBox, QPushButton, QHBoxLayout, QCheckBox, QScrollArea, QListWidget, QListWidgetItem, QAbstractItemView, QMenu
import time
import folium
from PyQt5.QtCore import QUrl
import random

ModuleNotFoundError: No module named 'Modele'

## III - Jeu de donnée <a class="anchor" id="Q3"></a>

Les données constituent des fichiers fournis par le logiciel GipsyX (logiciel développé par la NASA).

3 types de fichiers nous sont fournis.

- fichier `smooth.pos` qui contient les paramètres orbitaux d’un satellite (position et vitesse), ce qui donne par colonnes les variables : nom du repère (ici repère terrestre fixe), le nom du satellite, la date de la position du satellite en seconde, et ensuite les positions x, y et z du satellite en coordonnées cartésiennes suivit du vecteur vitesse selon ses trois composantes vx, vy et vz. 

- fichier `finalresiduals.out` qui correspond par colonne aux variables : la date d’acquisition des résidus en seconde depuis le 1er janvier 2000, le nom du satellite associé au nom de la station qui le mesure, le type de modèle, les valeurs résiduelles associées au satellite en m/s, l’élévation du satellite, l’azimut du satellite, l’élévation de la station et pour finir l’azimut de la station.

- fichier `smooth.tdp`. C’est un fichier de paramètre comme les paramètres de tropo ou d’accélération empirique (en sinus et cosinus) contenant : date mesure, apriori (ce qu’on estime), valeur du paramètre et écart-type associé.

## IV - Explications des différentes classes <a class="anchor" id="Q4"></a>

### a) Classe Modèle

   Elle permet de centraliser les données et les opérations nécessaires pour leur traitement, leur nettoyage, et leur analyse statistique. Elle récupère le type de données souhaité à l’aide de la classe LectureFichier, et grâce à la méthode `charge_donnees(self, type_fichier, nom_satellite, jour_debut, jour_fin, noms_stations=None)` permet d'afficher les valeurs voulues en fonction du calcul souhaité. Au cours de cette méthode, un premier nettoyage est effectué dans le jeu de donnée brute qui contient des lignes avec des valeurs complètement erronées. Pour la suite, d'autres méthodes de nettoyage pourraient être implémentées, pour trier les données de manière plus fines en fonction de la précision voulue. La variable `type_fichier` de cette méthode correspond aux 3 types de fichiers d'entrée expliquée dans la partie III - Jeu de donnée, à laquelle s'ajoute le type de fichier `position_stations` permettant de positionner les positions des stations au sol.


In [11]:
from LectureFichier import LectureFichier
import os
from pathlib import Path
from Modele import Modele
modele = Modele()

#--------Definition du chemin de la BDD-----------
current_dir = Path(os.getcwd())
chemin_donnees = current_dir / "BDD"
lecteur = LectureFichier(chemin_donnees)

##--------Lecture smooth.pos----------------------
nom_satellite = "JA3"
jour_debut = "2024-04-07"
jour_fin = "2024-04-09"
resultats = lecteur.lire_smooth_pos(nom_satellite, jour_debut, jour_fin)

# Sélectionner uniquement les colonnes 'jour', 'temps', 'x', 'y' et 'z'
colonnes_positions = ['jour', 'temps', 'x', 'y', 'z']
resultats_filtres = resultats[colonnes_positions]

print(resultats_filtres)

            jour      temps            x            y            z
0     2024-04-07  765709260  4093.952659  4594.584513 -4657.146493
1     2024-04-07  765709320  3794.755775  4549.637843 -4945.546398
2     2024-04-07  765709380  3483.357650  4493.265689 -5218.494107
3     2024-04-07  765709440  3160.626682  4425.748624 -5475.140175
4     2024-04-07  765709500  2827.468506  4347.393871 -5714.686379
...          ...        ...          ...          ...          ...
5380  2024-04-09  765989220 -4109.498754  4522.756255  4710.687247
5381  2024-04-09  765989280 -4096.181195  4217.776741  4996.327497
5382  2024-04-09  765989340 -4072.890863  3899.560407  5266.347183
5383  2024-04-09  765989400 -4039.814402  3569.009873  5519.905466
5384  2024-04-09  765989460 -3997.160664  3227.067287  5756.213272

[5385 rows x 5 columns]


In [4]:
    ## --------Lecture FinalResidual.out-------------
    final_residual_data = lecteur.lire_final_residual(nom_satellite='CS2', jour_debut='2024-04-07', jour_fin='2024-04-08', noms_stations=['SJVC'])
    print(final_residual_data)

Index(['2024-04-07', '2024-04-08'], dtype='object')
            temps satellite_station          modele  residuel  elevation_sat  \
0    7.657094e+08    CS2(1)-SJVC(1)  IonoFreeDorisD -0.000658        28.6942   
1    7.657094e+08    CS2(1)-SJVC(1)  IonoFreeDorisD  0.000120        28.0139   
2    7.657094e+08    CS2(1)-SJVC(1)  IonoFreeDorisD  0.000638        27.7292   
3    7.657095e+08    CS2(1)-SJVC(1)  IonoFreeDorisD -0.000504        31.9166   
4    7.657095e+08    CS2(1)-SJVC(1)  IonoFreeDorisD  0.000214        31.2266   
..            ...               ...             ...       ...            ...   
433  7.658826e+08    CS2(1)-SJVC(1)  IonoFreeDorisD -0.000375        31.6280   
434  7.658826e+08    CS2(1)-SJVC(1)  IonoFreeDorisD -0.000459        32.3741   
435  7.658827e+08    CS2(1)-SJVC(1)  IonoFreeDorisD  0.001294        27.8430   
436  7.658827e+08    CS2(1)-SJVC(1)  IonoFreeDorisD  0.000463        28.1524   
437  7.658826e+08    CS2(1)-SJVC(1)  IonoFreeDorisD  0.000808       

In [6]:
##---------Lecture smooth.tdp--------------------
smooth_tdp = lecteur.lire_fichiers_tdp("CS2", "2024-04-07", "2024-04-07")
print(smooth_tdp)

Index(['2024-04-07'], dtype='object')
              temps       apriori  valeur_parametre    ecart_type  \
0       765709260.0  7.657093e+08      1.594196e+07 -1.000000e+00   
1       765709260.0  0.000000e+00      9.527046e-14  8.742935e-14   
2       765709260.0  0.000000e+00      4.309197e-09  2.680147e-08   
3       765709260.0  0.000000e+00     -1.745539e-13  1.309751e-13   
4       765709260.0  0.000000e+00     -4.721713e-10  2.739601e-08   
...             ...           ...               ...           ...   
680395  765817200.0  0.000000e+00     -2.058202e-09  2.252777e-10   
680396  765817200.0  0.000000e+00      4.634516e-09  7.536299e-10   
680397  765817200.0  0.000000e+00      6.608545e-09  1.971923e-09   
680398  765817200.0  0.000000e+00     -1.038128e-11  9.907576e-10   
680399  765817200.0  2.041962e+00      4.696270e+00  2.698811e+00   

                                            nom_parametre  
0       Source.ID,Rogue,/home/snahmani/GipsyX-versions...  
1            

In [7]:
##-------Lecture station_data.txt------------
chemin_dossier = current_dir / "stations_data.txt"
lecture_fichier = LectureFichier(chemin_dossier)
df_stations = lecture_fichier.lire_position_stations('stations_data.txt')
print(df_stations)

          nom_complet   longitude   latitude abreviation
0             AJACCIO    8.762444  41.927472        AJAB
1           AMSTERDAM   77.571361 -37.798583        AMVB
2            AREQUIPA  -71.493000 -16.465778        ARFB
3               ARLIT    7.358583  18.782194        ARMA
4           ASCENSION  -14.332639  -7.916194        ASEB
..                ...         ...        ...         ...
95             WALLIS -176.179472 -13.265750        WALA
96           WETTZELL   12.880000  49.145000        WEUC
97         YARRAGADEE  115.346750 -29.046167        YASB
98        YELLOWKNIFE -114.479861  62.481111        YEMB
99  YUZHNO-SAKHALINSK  142.716778  47.029694        SAKB

[100 rows x 4 columns]


Dans cette même classe `Modele`, on trouve la méthode `obtenir_stations_final_residual`, qui intervient lors de la saisie utilisateur. En effet, comme expliqué dans la documentation utilisateur, l'utilisateur va pouvoir choisir le satellite qu'il veut étudier en particulier. Suite à cela, il charge les stations associées, car en effet, chaque satellite est vu par un nombre de station limité. Etant donné que l'utilisateur a la possibilité de choisir une station, il faut que cette dernière soit choisie en fonction du satellite choisi. C'est donc pour cela que méthode a été créée.

In [12]:
stations = modele.obtenir_stations_final_residual("S3B", "2024-04-07", "2024-04-09")
print(stations, len(stations))

Index(['2024-04-07', '2024-04-08', '2024-04-09'], dtype='object')
['ADHC', 'AMVB', 'ASEB', 'BEMB', 'BETB', 'CIDB', 'COBB', 'CRRC', 'DIOB', 'DJIB', 'EVEB', 'GAVC', 'GONC', 'GR4B', 'GRFB', 'HBMB', 'HEMB', 'HROC', 'JIWC', 'KEYC', 'KIVC', 'KOLB', 'KRWB', 'LAOB', 'LICB', 'MAIB', 'MALC', 'MANB', 'MAVC', 'MEUB', 'MIAB', 'MLAC', 'MNAC', 'MSPB', 'NOXC', 'OWFC', 'PAUB', 'PDOC', 'REVC', 'RIMC', 'RISC', 'ROBC', 'SARC', 'SCSC', 'SJVC', 'SOFC', 'STKC', 'SVBC', 'TLSB', 'TRJB', 'YASB', 'YEMB'] 52


### b) Classe Controleur

   Concernant le contrôleur, il gère les interactions entre le modèle et la vue. Il effectue le chargement des données selon les sélections de l’utilisateur c’est à dire qu’il déclenche le chargement et traitement des fichiers en fonction de la période ou des stations choisies. Il gère les événements de visualisation en répondant aux actions de l’utilisateur pour mettre à jour les vues (ex., choisir une station, modifier la période). Pour finir, il permet de gérer les configuration dynamique en intégrant la modularité pour permettre de nouvelles fonctionnalités et nouveaux affichages sans changements majeurs dans le code.

- `ajouter_actions(self)` : Cette méthode permet de gérer l'action de tous les boutons présents sur l'interface graphique. Cela correspond aux boutons `Saisie-Utilisateur`, `Statistiques`, `Visualisation` (comprenant `Skyplot-Station` et `Skyplot - Satellite`) et `Redémarrer`. Ces méthode permet ainsi de renvoyer l'action du clique sur ces boutons, aux méthodes respectifs associés permettant de réaliser l'action prévue pour le bouton.

- `recuperer_donnees_base(self)`: 
